In [1]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Flatten

Using TensorFlow backend.


In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print(y_train)

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
print(y_train)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
[[6]
 [9]
 [9]
 ...
 [9]
 [1]
 [1]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [10]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 224, 224, 16) 448         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_29 (BatchNo (None, 224, 224, 16) 64          conv2d_32[0][0]                  
__________________________________________________________________________________________________
activation_225 (Activation)     (None, 224, 224, 16) 0           batch_normalization_29[0][0]     
____________________________________________________________________________________________

In [8]:
#my test network
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

from keras.applications.resnet50 import ResNet50
from keras.models import Sequential

resnetModel = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = resnetModel.output
x = Flatten()(x)

#resnetModel = ResNet50(input_shape =(32,32,3), weights ='imagenet', pooling = 'avg', include_top = False )
#last_featMap = resnetModel.output
#flatten_featMap = Flatten()(last_featMap)
#output = Dense(num_classes, activation='softmax')(flatten_featMap)
#MyResnetModel = Model(inputs = resnetModel.input, outputs = output)
resnetModel = Sequential()
resnetModel.add(ResNet50(include_top = False ,weights = 'imagenet', input_shape =(224,224,3), pooling = 'avg', classes = 10 ))
#resnetModel.add(ResNet50(weights = 'imagenet', pooling = 'avg', include_top = False ))
resnetModel.add(Flatten())
resnetModel.add(Dense(512))
resnetModel.add(Activation('relu'))
resnetModel.add(Dropout(0.25))
resnetModel.add(Dense(num_classes))
resnetModel.add(Activation('softmax'))


ValueError: Input 0 is incompatible with layer flatten_4: expected min_ndim=3, found ndim=2

In [14]:
#my test network
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

from keras.applications.resnet50 import ResNet50
from keras.models import Sequential

resnetModel = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(32,32,3))
x = resnetModel.output
x = Flatten()(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(num_classes, activation='softmax', name='softmax')(x)

resnetModel50 = Model(inputs=resnetModel.input, outputs=output_layer)

In [15]:
from sklearn.utils import shuffle
def my_generator(x, y, batch_size):
    while True:
        for idx in range(0, len(x), batch_size): # 讓 idx 從 0 開始，一次增加 batch size。假設 batch_size=32, idx = 0, 32, 64, 96, ....
            batch_x, batch_y = x[idx:idx+batch_size], y[idx:idx+batch_size]
            yield batch_x, batch_y
        x, y = shuffle(x, y) # loop 結束後，將資料順序打亂再重新循環

In [16]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

#train_generator = my_generator(x_train, y_train, batch_size) # 建立好我們寫好的 generator

#model.compile(loss='categorical_crossentropy',
#              optimizer=Adam(),
#              metrics=['accuracy'])

resnetModel50.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

#history = model.fit_generator(train_generator,
#                    steps_per_epoch=int(len(x_train)/batch_size), # 一個 epochs 要執行幾次 update，通常是資料量除以 batch size
#                    epochs=epochs,
#                    verbose=1,
#                    validation_data=(x_test, y_test))

history = resnetModel50.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

#score = model.evaluate(x_test, y_test, verbose=0)
score = resnetModel50.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 3424s 68ms/step - loss: 1.1975 - accuracy: 0.6210 - val_loss: 3.7196 - val_accuracy: 0.1000
Epoch 2/30
50000/50000 [==============================] - 3277s 66ms/step - loss: 0.7094 - accuracy: 0.7605 - val_loss: 1.7173 - val_accuracy: 0.4655
Epoch 3/30
50000/50000 [==============================] - 3274s 65ms/step - loss: 0.6494 - accuracy: 0.7888 - val_loss: 2.1626 - val_accuracy: 0.4485
Epoch 4/30
50000/50000 [==============================] - 3283s 66ms/step - loss: 1.0444 - accuracy: 0.6737 - val_loss: 1.5134 - val_accuracy: 0.5746
Epoch 5/30
50000/50000 [==============================] - 3278s 66ms/step - loss: 0.7893 - accuracy: 0.7395 - val_loss: 0.8908 - val_accuracy: 0.7052
Epoch 6/30
50000/50000 [==============================] - 3272s 65ms/step - loss: 0.6010 - accuracy: 0.7999 - val_loss: 1.1086 - val_accuracy: 0.6607
Epoch 7/30
50000/50000 [==========================

KeyboardInterrupt: 

## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠訓練!

最後一天的作業請閱讀這篇[非常詳盡的文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet

## 進階挑戰!
有志以成為機器學習工程師為目標的同學們可以參加這個挑戰，這[網站](https://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#43494641522d3130)記錄了歷年來 Cifar-10 中排名最高的論文，請試著閱讀論文並撰寫出相對應的程式碼，復現出論文的結果。

這樣的能力在機器學習領域中是非常重要的，具備閱讀他人論文並實現的能力，可為自己在履歷上增添不少分數，當然難度也相當高，若是不透徹了解文章內容或是程式能力不夠紮實，可是不能復現別人辛苦的研究成果的喔! 就請各位同學好好努力，往自己的機器學習之路邁進吧:)